In [1]:
from imports import *
from exactDiagMethods import *
from globalMethods import  *
from vqteMethods import *



In [2]:
def main():
    # Initialize the simulation parameters
    N = 1
    gamma_L= 1.0
    gamma_R = 1.0
    eps = 1
    mu_L= 3
    mu_R= 3
    T_L = 10   
    T_R= 10
    beta_L=1/T_L
    beta_R=1/T_R
    F_L = 1/(np.exp(beta_L*(eps-mu_L))+1)
    F_R = 1/(np.exp(beta_R*(eps-mu_R))+1)
    J = 1
    
    #Layer = 2

    eps_list = []

    for i in range(N):
        eps_list.append(1)

  
    time = 6.0
    dt = 0.05
    nt = int(time/dt)

    

    exact_diag_ham = build_exact_diag_hamiltonian(J, eps_list)

    #print("Exact Ham", exact_diag_ham)

    ham_real, ham_imag = hamiltonian_generation(N, eps_list, gamma_L, gamma_R, F_L, F_R, J)

    print("Exact Diag Ham: ", exact_diag_ham)
 
    
    
    print("VQTE Hamiltonian Matrix: ", (ham_imag.to_matrix()+ham_real.to_matrix()))
    # Generate initial states
    vqte_init_state, exact_diag_init_state, ansatz, init_param_values = build_initial_states(ham_real)



    exact_diag_results = perform_exact_diag(gamma_L,F_L, gamma_R, F_R, dt, nt, exact_diag_init_state,exact_diag_ham, N)

 
    vqte_results = perform_vqte(ham_real, ham_imag, vqte_init_state, dt, nt, ansatz, init_param_values)
    
    # # Output results
    output_results(vqte_results, exact_diag_results, time, nt)

In [3]:
main()



Exact Diag Ham:  [[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
VQTE Hamiltonian Matrix:  [[ 1.09966799+0.j          0.        +0.j          0.        +0.j
  -1.25      -0.09966799j]
 [ 0.        +0.j          2.        +0.j         -0.25      +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.25      +0.j          0.        +0.j
   0.        +0.j        ]
 [-1.25      +0.09966799j  0.        +0.j          0.        +0.j
   0.90033201+0.j        ]]
Exact step  1  out of 120
Exact step  2  out of 120
Exact step  3  out of 120
Exact step  4  out of 120
Exact step  5  out of 120
Exact step  6  out of 120
Exact step  7  out of 120
Exact step  8  out of 120
Exact step  9  out of 120
Exact step  10  out of 120
Exact step  11  out of 120
Exact step  12  out of 120
Exact step  13  out of 120
Exact step  14  out of 120
Exact step  15  out of 120
Exact step  16  out of 120
Exact step  17  out of 120
Exact step  18  out of 120
Exact step  19  out of 120
Exact step  20  out of 120
Exact step  21  ou

NameError: name 'num_op_1q_matrix' is not defined